# RNAseq Analysis Module

## Practical session 7: Differential analysis with high-throughput technologies (microarrays and RNAseq)
Thrusday, the 3rd of December, 2020   
Claire Vandiedonck and Sandrine Caburet - 2020 

          0. R session   
          1. Getting your microarrays and RNASeq data   
          2. Differential Expression analysis with microarray experiments 
          3. Differential analysis with RNA Seq experiments (count data)
          4. Compare RNASeq and microarray DE results

## 0 - R session

Start R and set your working directory:

In [ ]:
#setwd() # specify the "path" to your working directory within the brackets
getwd()

Call required packages (which are already installed in this environment). Some warnings can arise due to changes in R versions, but as a general rule the functions in these packages will still work.

In [ ]:
library(gplots) # used for advanced graphs
library(locfit) # used for regression analysis
library(limma) # used for expression microarrays differential analysis (now also for RNASeq: we will stick to microarray data here)
library(DESeq) # used for expression RNASeq differential analysis

However, in your scripts, you might not use this environment. When you are not sure about whether a package has been installed, you can use this syntax to call the package if it is already present, otherwise install it and call it.

Some packages are not available via the R CRAN package repository, specifically the ones used in bioinformatic analyses, but via Bioconductor. In order to install a package from Bioconductor, you can proceed as follows:

This command will return the R and R-packages versions used in your R session. You can check that indeed the packages called in a previous command are present:

In [ ]:
sessionInfo()

## I - Getting your microarrays and RNASeq data

Remember that here, our microarray and RNAseq data are stored in different folders:

In [ ]:
path_to_microarray = "/srv/data/meg-m2-rnaseq/analysis_data/"
path_to_RNAseq = "~/m2meg-rnaseq-tp3to5-bash/Results/Counts/"
dataArrays <- read.table(paste0(path_to_microarray, "Microarrays_logValues_parapsilosis.txt"), header = T, row.names = 1)
# Merge all RNAseq counts data
counts_names = c("Hypoxia_1", "Normoxia_1", "SRR352261", "SRR352264", "SRR352266", 
                 "SRR352267", "SRR352270", "SRR352273", "SRR352274", "SRR352276")
# Using the sample sheet completed on Monday
new_names = c("Hypoxia_1", "Normoxia_1", "Normoxia_2", "Normoxia_3", "Normoxia_4", 
                 "Hypoxia_2", "Hypoxia_3", "Normoxia_5", "Hypoxia_4", "Normoxia_6")
suffix = "_gene_counts.gff"
N = length(counts_names)
# One can merge iteratively all dataframes, however, it is very time- and memory-consuming
# The best move is to initialize the quantity of memory you need from the start, and fill it with data
# since slicing in data frames is cheap
convert_gff_id_to_gene_name <- function(rnames) {
    as.vector(sapply(rnames, function (x) gsub("ID=", "", x)))
}
rows <- as.factor(row.names(dataArrays))
# 0 is the default expression count of a gene which has no reads
ls = lapply(1:N, function(x) rep(0, length(rows)))
names(ls) <- new_names
dataRNAseq <- as.data.frame(ls, row.names=rows)
# Now fill the initialized dataframe with data
for (i in 1:N) {
    datacounts <- read.table(paste0(path_to_RNAseq, counts_names[i], suffix), header=T, row.names=9, stringsAsFactors=T)
    rownames(datacounts) <- convert_gff_id_to_gene_name(row.names(datacounts))
    dataRNAseq[row.names(datacounts),new_names[i]] <- datacounts[,ncol(datacounts)]
}
# reorder the lines in the same order as in the microarray data
dataRNAseq <- dataRNAseq[rows,]
rownames(dataRNAseq) <- rows
# look at the R objects you generated
head(dataArrays)
head(dataRNAseq)
tail(dataArrays)
tail(dataRNAseq)

Note that genes listed in both files are identical, with the same order. This can be automatically checked with the following command line counting all occurences that are identical (TRUE) and not (FALSE):

In [ ]:
table(row.names(dataArrays) == row.names(dataRNAseq))

In [ ]:
# rename colnames
colnames(dataArrays) <- c("log(H/N)_rep1", "log(H/N)_rep2", "log(H/N)_rep3", "log(H/N)_rep4")
colnames(dataRNAseq) <- c("countN_rep1", "countN_rep2", "countN_rep3", "countN_rep4", "countN_rep5", "countN_rep6", "countH_rep1", "countH_rep2","countH_rep3", "countH_rep4")
#check the structure of the new R objects
str(dataArrays)
str(dataRNAseq)

## II - Differential Expression analysis with microarray experiments

### Q1. Were microarrays normalised?

A1. To answer this first question, we will generate two figures of the distribution of the log2(intensity ratios):

#### A1.1. Histogram 

In [ ]:
xValues  <- seq(floor(min(dataArrays)), ceiling(max(dataArrays)), by = 0.1) 
par(mfrow = c(2,2))
for (i in 1:ncol(dataArrays)) {
        hist(dataArrays[,i], breaks = xValues, 
        xlab = "log2(H/N) value", 
        main = paste("log2(H/N) values\n array #", i))
}
par(mfrow = c(1,1))

#### A1.2. One boxplot per array

In [ ]:
boxplot(dataArrays,outline = F, main = "Boxplot of log2FC distributions for each microarray replicate", ylab = "log2FC")

Some descriptive values of the distribution can also be generated automatically:

In [ ]:
summary(dataArrays)

### Q2. Are data correlated between replicates?

A2. We generate a matrix of pairwise (Pearson's R) correlations, display the pairwise correlation plots, and add the *p*-values of the Pearson statistical tests:

In [ ]:
panel.cor <- function(x, y, digits=2, prefix2="", cex.cor) {
     usr <- par("usr"); on.exit(par(usr)) 
     par(usr = c(0, 1, 0, 1)) 
     r <- abs(cor(x, y)) 
     txt <- format(c(r, 0.123456789), digits = digits)[1] 
     txt <- paste(prefix2, txt, sep = "") 
     if (missing(cex.cor)) cex <- 0.8/strwidth(txt) 
     test <- cor.test(x,y) 
     Signif <- symnum(test$p.value, corr = FALSE, na = FALSE, 
     cutpoints = c(0, 0.001, 0.01, 0.05, 0.1, 1),
     symbols = c("***", "**", "*", ".", " ")) 
     text(0.5, 0.5, txt, cex = cex * r) 
     text(.8, .8, Signif, cex = cex, col = 2) 
      }
pairs(as.matrix(dataArrays), labels = colnames(dataArrays), lower.panel = panel.smooth, upper.panel = panel.cor, main = "pairwise correlation of microarray replicates")

### Q3. Is there a relationship between the mean and the standard deviation of the log(H/N)? Is this a problem?

Compute and plot some descriptive statistic for each gene among the 4 replicates:

In [ ]:
# mean value for log(H/N) replicates for each gene
averageLog <- apply(dataArrays, 1, mean) 
# standard deviation for log(H/N) replicates for each gene
sdValues <- apply(dataArrays, 1, sd)  
plot(sdValues, averageLog, 
        main = "C. parapsilosis - Microarrays dataset",
        xlab = "Standard deviations", ylab = "Average log2FC",
        pch = 20)
abline(h = 2, col = "red", lty = "dashed")
abline(h = -2, col = "green", lty = "dashed")

### Q4. What is the effect of Bonferroni correction? How many genes are up- or down-regulated?

Perform Differential Expression analysis with a Student t-test:

In [ ]:
# t-values
tval <- averageLog/(sdValues/sqrt(ncol(dataArrays))) 

Compute t-threshold for Type I error $\alpha = 5\%$

In [ ]:
alpha=0.05
critical_0.05_t <- qt(1 - alpha/2, 4 - 1)  
# Number of genes with pval < 5%
length(tval[which(abs(tval) > critical_0.05_t)]) 
# Number of genes upregulated with pval < 5%
length(tval[which(tval > critical_0.05_t)]) 
# Number of genes downregulated with pval < 5%
length(tval[which(tval < -critical_0.05_t)]) 

Same procedure with $\alpha = 1\%$:

In [ ]:
alpha=0.0001
critical_0.0001_t <- qt(1 - alpha/2, 4 - 1)
length(tval[which(abs(tval) > critical_0.0001_t)])
length(tval[which(tval > critical_0.0001_t)])
length(tval[which(tval < -critical_0.0001_t)])

Same Student test with Bonferroni correction and Type I error at $\alpha=5\%$:

In [ ]:
alpha=0.05
critical_bonf_t <- qt(1 - (alpha/(2*length(tval))), 4 - 1) 
length(tval[which(abs(tval) > critical_bonf_t )])
length(tval[which(tval > critical_bonf_t )])
length(tval[which(tval < -critical_bonf_t )])
## Genes which t-values are above, resp. below, the critical threshold at alpha=5% adjusted using the Bonferroni correction
upGenes.t <- names(tval[which(tval > critical_bonf_t )])
downGenes.t <- names(tval[which(tval < -critical_bonf_t )])

Now, plot the t-values and fold changes (FC), and highlight the aforementioned DE genes:

In [ ]:
plot(averageLog, tval,
        main = "C. parapsilosis - Microarrays dataset",
        ylab = "tval (Student parameter)", xlab = "LogFC (average value)",
        pch = 20)
abline(h = 0, col = "black")
abline(v = 0, col = "black")
abline(h = critical_bonf_t, col = "red", lty = "dashed")
abline(h = -critical_bonf_t, col = "green", lty = "dashed")
points(averageLog[upGenes.t],tval[upGenes.t], col = "red", pch = 20)
points(averageLog[downGenes.t],tval[downGenes.t], col = "green", pch = 20)

### Q5. Do the FC values change using limma? How do the moderated t-statistics of limma behave compared to Student t-statistics and to the initial standard deviation? Is that satisfactory?

Perform Differential Expression analysis using **limma**:

In [ ]:
# Linear model estimation
fit <- lmFit(dataArrays)
# Bayesian statistics
limmaRes <- eBayes(fit)
limmaTable <- topTable(limmaRes, number = nrow(dataArrays))
# Sort genes according to their initial order in dataArrays
limmaRes2 <- limmaTable[row.names(dataArrays),]

Compare logFC values obtained with **limma**:

In [ ]:
plot(limmaRes2[, "logFC"], averageLog, pch = 20, xlab = "logFC calculated with limma", ylab = "LogFC (average value)")
cor.test(limmaRes2[, "logFC"], averageLog)

Compare t-values from **limma** with the t-values obtained with the Student's test with regard to FC:

In [ ]:
plot(averageLog, tval,
        main = "C. parapsilosis - Microarrays dataset",
        ylab = "t (Student in black and limma in purple)", xlab = "LogFC (average value)",
        pch = 20)
points(averageLog, limmaRes2[,"t"], pch = 20, col = "purple")
abline(h = 0, col = "black")
abline(v = 0, col = "black")

Compare t-statistic from **limma** with the t-values obtained with the Student's test with regard to (gene expression value) standard deviation:

In [ ]:
plot(sdValues, tval,
        main = "C. parapsilosis - Microarrays dataset",
        ylab = "t (Student in black and LIMMA in purple)", xlab = "Standard deviation",
        pch = 20)
points(sdValues, limmaRes2[,"t"], pch = 20, col = "purple")
abline(h = 0, col = "black")

### Q6. With **limma** and an adjusted type I error of $\alpha=0.0001$, how many genes are DE? up-regulated? down-regulated? Are they the same genes as the ones found using the Student's tests?

In [ ]:
# genes differentially expressed with microarrays
alpha=0.0001
## Number of genes with an adjusted p-value < alpha
length(which(limmaRes2$adj.P.Val < alpha))
## Number of genes with an adjusted p-value < alpha *and a positive log(FC)*
dim(limmaRes2[which(limmaRes2$adj.P.Val < alpha & limmaRes2$logFC > 0),])[1]
upGenes.limma <- limmaRes2[which(limmaRes2$adj.P.Val < alpha & limmaRes2$logFC > 0),]
downGenes.limma <- limmaRes2[which(limmaRes2$adj.P.Val < alpha & limmaRes2$logFC < 0),]

In order to visualize the genes according to their DE *p*-value and FC, we will generate a volcano plot: 

In [ ]:
plot(-log10(limmaRes2$adj.P.Val) ~ limmaRes2$logFC, pch = "", xlim = c(-6,6),ylim = c(0,7),
     xlab = "",ylab = "",bty = "n",xaxt = "n", yaxt = "n"  )
title("Hypoxia versus Normoxia", font.main = 1, cex.main = 0.9)
axis(1, at = -6:6, tcl = -0.5,cex = 0.7, labels = F )
mtext(-6:6,side = 1,line = 1,at = -6:6, cex = 0.7)
axis(2, at = 0:7, tcl = -0.2, cex = 0.7, labels = F )
mtext(0:7,side = 2,line = 0.5, at = 0:7, cex = 0.7)
## Color in grey genes
points(-log10(limmaRes2$adj.P.Val) ~ limmaRes2$logFC, pch = 16, cex = 0.5, col = "grey")
## Override the previous colouring for some genes according to the fact they are DE
points(-log10(upGenes.limma$adj.P.Val) ~ upGenes.limma$logFC,pch = 16, cex = 0.5, col = "red")
points(-log10(downGenes.limma$adj.P.Val) ~ downGenes.limma$logFC, pch = 16,cex = 0.5, col = "green")
mtext("log2(FC)", side = 1, line = 2, cex = 0.8)
mtext("-log10 (adjusted p-value)", side = 2, line = 1.5, cex = 0.8)

Let's compare these up- and down-regulated genes with the ones obtained through the Student's tests:

In [ ]:
length(intersect(downGenes.t,row.names(upGenes.limma)))
length(setdiff(row.names(downGenes.limma), upGenes.t))
length(setdiff(downGenes.t,row.names(upGenes.limma)))
length(intersect(downGenes.t,row.names(downGenes.limma)))
length(setdiff(row.names(downGenes.limma), downGenes.t))
length(setdiff(downGenes.t,row.names(downGenes.limma)))

Compare ranks of genes in both analyses (**limma** and Student's test):

In [ ]:
sortedtval <- names(sort(abs(tval),decreasing = TRUE))
sortedtval <- data.frame(sortedtval,1:length(sortedtval), stringsAsFactors = F)
names(sortedtval)[2] <- "rank.t"   
limmaTable$rank.limma <- 1:length(row.names(limmaTable))
limmaTable$gene <- row.names(limmaTable)

Combine all microarray results in a single dataframe:

In [ ]:
dataArrays.analyzed <- data.frame(dataArrays,averageLog,sdValues,tval)
dataArrays.analyzed$gene <- row.names(dataArrays.analyzed)
dataArrays.analyzed <- merge(dataArrays.analyzed, sortedtval, by.x = "gene", by.y = "sortedtval", all = T, sort = F)
dataArrays.analyzed <- merge(dataArrays.analyzed, limmaTable, by = "gene", all = T, sort = F)

Plot ranks of genes in the Student's analysis versus **limma**:

In [ ]:
plot(dataArrays.analyzed$rank.limma ~dataArrays.analyzed$rank.t, pch = 16, cex = 0.35, xlab = "rank.t", ylab = "rank.limma")

Save the final dataframe containing the results in your working directory:

In [ ]:
write.table(dataArrays.analyzed, quote = F, sep = "\t", file = "Microarrays_diffAnalysis.txt", row.name = F)

## III - Differential analysis with RNA Seq experiments (count data)

### Q7. Are the RNASeq data normalised?

Compute the logFC values using the read counts:

In [ ]:
# Mean values for normoxic and hypoxic replicates
meanNcounts = apply(dataRNAseq[,1:6], 1, mean)
meanHcounts = apply(dataRNAseq[,7:10], 1, mean)
# logFC (raw data)
logFC = log2((meanHcounts + 1)/(meanNcounts + 1))

Let's plot the distribution of log(FC) (raw data):

In [ ]:
hist(logFC, nclass = 100, main = "logFC (H/N) distribution \n(raw data)", xlab = "log(H/N) value", freq=F)
abline(v = 0, col = "red")

### Q8. What are the library effect sizes? Are they consistent with the library sizes?

Let's normalise the read count data using DESeq:

In [ ]:
# create metadata: qualitative and quantitative information about the data different from the expression values
conds = c(rep("N",6), rep("H",4))
expDesign = data.frame(row.names = colnames(dataRNAseq), 
                       condition = conds,
                       libType   = rep("RNAseq", 10))

In [ ]:
# create a countDataSet: specific R object in DESeq to manipulate expression datasets
cds = newCountDataSet(dataRNAseq, expDesign$condition)

In [ ]:
# Estimate normalization factors
cds = estimateSizeFactors(cds)
# Inspect size factors
sizeFactors(cds)
summary(sizeFactors(cds))

In [ ]:
apply(dataRNAseq, 2, sum)
sort(sizeFactors(cds))
sort(apply(dataRNAseq, 2,  sum))

In [ ]:
cor.test(sizeFactors(cds),apply(dataRNAseq, 2,  sum), method="s")
plot(sizeFactors(cds),apply(dataRNAseq, 2,  sum))

### Q9. Are you satisfied by the normalisation performed?

In [ ]:
# Get normalized count values
cdsNorm <- counts(cds, normalized = TRUE)
# Mean values
meanNcountsNorm <- apply(cdsNorm[,1:6], 1, mean)
meanHcountsNorm <- apply(cdsNorm[,7:10], 1, mean)
# standard deviation (sd) values for log(H/N) replicates
sdNcountsNorm   <- apply(cdsNorm[,1:6], 1, sd)
sdHcountsNorm   <- apply(cdsNorm[,7:10], 1, sd)
# logFC (after normalization)
logFCNorm = log2((meanHcountsNorm + 1)/(meanNcountsNorm + 1))

In [ ]:
hist(logFCNorm, nclass = 100, main = "logFC (H/N) distribution \n(normalized data)", xlab = "log(H/N) value")
abline(v = 0, col = "red")

### Q9. How many genes are differentially expressed at the adjusted type I error of 0.0001? up-regulated ? down-regulated?

Let's perform the DE analysis with DESeq! First we need to estimate the gene dispersions:

In [ ]:
## Takes some time
cds = estimateDispersions(cds)
# To inspect the intermediate steps, a fitInfo object is stored
# which contains the per genes estimate, the fitted curve
# and the values that will subsequently be used for inference.
str(fitInfo(cds))

Let's visualise the gene dispersions: we can observe that genes with low expression exhibit higher levels of variability (dispersion).

In [ ]:
plotDispEsts(cds)

The function which performs the differential analysis by itself (this step takes a few seconds to be performed, and returns a dataframe similar to the one we merged for the microarray data):

In [ ]:
res <- nbinomTest(cds, "N", "H")

In [ ]:
# reorder according to the adjusted p-values
res_ordered <- res[order(res$padj),]

# filter according to p-values < 0.0001
resSig <- res[ res$padj < 0.0001, ]
nrow(resSig)
table(resSig$log2FoldChange > 0)

# the most significant:
head( resSig[ order(resSig$pval), ] )
head( resSig[ order( resSig$foldChange, -resSig$baseMean ), ] )
head( resSig[ order( -resSig$foldChange, -resSig$baseMean ), ] )

Let's plot a heatmap of the normalised expression values of the most significant genes:

In [ ]:
top_DGE_rnaseq <- cdsNorm[which(row.names(cdsNorm) %in% resSig$id),]
dim(top_DGE_rnaseq)
coolmap(top_DGE_rnaseq, col = "yellowblue", cexCol = 0.6, cexRow = 0.2 )

As done for the microarray data, let's save the differential analysis results:

In [ ]:
write.table(res, "RNASeq_DiffAnalysis.txt", row.name = F, quote = F, sep = '\t')

## IV - Compare RNASeq and microarray DE results

In [ ]:
# Merge the DE results from microarray and RNASeq data:
arrays_rnaseq <- merge(dataArrays.analyzed, res, by.x="gene",by.y="id",sort=F,all=T)
write.table(arrays_rnaseq, "Arrays_RNASeq_DiffAnalysis.txt", row.name = F, quote = F, sep = '\t')

### Q10. Which are the 20 differentially expressed genes we would like to study further? 

Use either R or a calculator to select your candidate genes, according to what you deem important (p-values, FC, ...)

In [ ]:
## TODO

An example:

In [ ]:
Ntop=20
alpha=0.0001
# Commands to extract your 20 top genes according to your criteria:
both_ <- subset(arrays_rnaseq, adj.P.Val < alpha & padj < alpha)
dim(both_)
head(both_)
abs(both_$log2FoldChange)
sort(abs(both_$log2FoldChange))
sort(abs(both_$log2FoldChange), decreasing=T)
sort(abs(both_$log2FoldChange), decreasing=T)[Ntop]
FC_threshold <- sort(abs(both_$log2FoldChange), decreasing=T)[Ntop]
subset(both_, abs(log2FoldChange) >= FC_threshold)
our20genes <- subset(both_, abs(log2FoldChange) >= FC_threshold)
write.table(our20genes, "our20genes.txt", row.name = F, quote = F, sep = '\t')

Some cleaning /!\ Do not run if you want to keep the DE results obtained in the TPs!

In [ ]:
file.remove("Microarrays_diffAnalysis.txt")
file.remove("RNASeq_DiffAnalysis.txt")
file.remove("Arrays_RNASeq_DiffAnalysis.txt")
file.remove("our20genes.txt")


---

__*About jupyter notebooks*__

- To add a new cell, click on the "+" icon in the menu bar above
- You can "click and drag" to move a cell up or down
- You choose the type of cell in the dropdow menu in the menu bar above
    - 'Code' to enter command lines to be executed 
    - 'Markdown' cells to add text, that can be formatted with some characters 
- To execute a 'Code' cell, press SHIFT+ENTER or click on the "play" icon 
- To display a 'Markdown' cell, press SHIFT+ENTER or click on the "play" icon  
- To modify a 'Markdown'cell, double-click on it
    
---